In [33]:
import pandas as pd
import json

In [34]:
output_file = pd.read_csv('./data/all_train_pairs_combined_results_finetunedModel.csv')

/var/folders/50/ddkl_7r93pv5mq0_h58b4lv80000gn/T/ipykernel_97834/571563597.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  output_file = pd.read_csv('./data/all_train_pairs_combined_results_finetunedModel.csv')


In [35]:
output_file.label.value_counts()

label
a    40014
b    39375
d    27747
c    25083
e     1422
f     1404
g     1332
Name: count, dtype: int64

In [40]:
# Check one

a = output_file[output_file.label == 'e']
a.iloc[0].logits

"[{'letter': 'e', 'logit': 27.9375, 'probability': 0.9520723819732666}, {'letter': 'd', 'logit': 24.734375, 'probability': 0.03868747875094414}, {'letter': 'b', 'logit': 22.234375, 'probability': 0.0031756616663187742}, {'letter': 'a', 'logit': 22.03125, 'probability': 0.00259189959615469}, {'letter': 'c', 'logit': 21.65625, 'probability': 0.00178138492628932}, {'letter': 'g', 'logit': 21.09375, 'probability': 0.0010150026064366102}, {'letter': 'f', 'logit': 20.6875, 'probability': 0.0006761374534107745}]"

## Data Cleaning

In [41]:
import ast
import numpy as np

# First, let's standardize the answer choices
valid_choices = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

# Function to convert string representation of logits to numpy array
def extract_logits(logits_str):
    '''
    The resulting logits will be stored as numpy arrays with shape (4,), 
    corresponding to the logits for ['a', 'b', 'c', 'd', 'e', 'f', 'g'] in that order.
    '''
    try:
        # Convert string to list of dicts
        logits_list = ast.literal_eval(logits_str)
        
        # Create a dictionary to store logits for valid choices
        logits_dict = {item['letter'].lower(): item['logit'] for item in logits_list 
                      if item['letter'].lower() in valid_choices}
        
        # Create array in consistent order
        return np.array([logits_dict.get(choice, 0.0) for choice in valid_choices])
    except:
        return np.zeros(len(valid_choices))

# Process the dataframe
def process_dataset(df):
    # Create a copy to avoid modifying original
    processed_df = df.copy()
    
    # Convert predicted answers to lowercase
    processed_df['predicted_answer'] = processed_df['predicted_answer'].str.lower()
    
    # Filter out invalid predictions
    processed_df = processed_df[processed_df['predicted_answer'].isin(valid_choices)]
    
    # Convert logits to numpy arrays
    processed_df['logits'] = processed_df['logits'].apply(extract_logits)
    
    # Drop the original sorted_logits column
    # processed_df = processed_df.drop('logits', axis=1)
    
    return processed_df

# Process the dataset
cleaned_output = process_dataset(output_file)

cleaned_output.head(100)

,question_text,question_image,answer_choices,context_image,context_image_caption,text_context,label,predicted_answer,logits
0,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,Most fossils are preserved by one of five proc...,d,d,"[25.40625, 22.484375, 25.484375, 27.265625, 0...."
1,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",textbook_images/erosion_and_deposition_by_flow...,FIGURE 10.5 How a Waterfall Forms and Moves. W...,Most fossils are preserved by one of five proc...,d,d,"[24.953125, 22.390625, 24.984375, 26.9375, 0.0..."
2,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/cycle_rock_6723.png,The diagram shows types of rocks and rock form...,Most fossils are preserved by one of five proc...,d,d,"[25.109375, 22.71875, 25.453125, 27.328125, 0...."
3,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,1. What is the traditional definition of gravi...,d,d,"[25.359375, 22.578125, 25.140625, 27.234375, 0..."
4,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",textbook_images/erosion_and_deposition_by_flow...,FIGURE 10.5 How a Waterfall Forms and Moves. W...,1. What is the traditional definition of gravi...,d,d,"[24.8125, 22.40625, 24.71875, 26.953125, 0.0, ..."
...,...,...,...,...,...,...,...,...,...
95,How many types of rocks are shown in the diagram?,question_images/erosion_6850.png,"a. 2, b. 1, c. 4, d. 3",teaching_images/parts_ocean_floor_9206.png,This diagram shows an abbreviated version of u...,Sedimentary rocks form in two ways. Particles ...,d,a,"[27.078125, 23.171875, 24.203125, 25.15625, 0...."
96,How many types of rocks are shown in the diagram?,question_images/erosion_6850.png,"a. 2, b. 1, c. 4, d. 3",teaching_images/erosion_8064.png,The diagram shows how a waterfall is formed by...,Index fossils are commonly used to match rock ...,d,a,"[27.765625, 22.40625, 24.078125, 25.265625, 0...."
97,How many types of rocks are shown in the diagram?,question_images/erosion_6850.png,"a. 2, b. 1, c. 4, d. 3",textbook_images/erosion_and_deposition_by_flow...,FIGURE 10.5 How a Waterfall Forms and Moves. W...,Index fossils are commonly used to match rock ...,d,a,"[27.28125, 22.6875, 24.296875, 25.484375, 0.0,..."
98,How many types of rocks are shown in the diagram?,question_images/erosion_6850.png,"a. 2, b. 1, c. 4, d. 3",teaching_images/parts_ocean_floor_9206.png,This diagram shows an abbreviated version of u...,Index fossils are commonly used to match rock ...,d,a,"[27.421875, 22.671875, 24.375, 25.4375, 0.0, 0..."


In [45]:
# Verify the results
print("Unique predictions after cleaning:")
print(cleaned_output.predicted_answer.value_counts())
print("\nSample logits array:")
print(cleaned_output['logits'].iloc[0])

Unique predictions after cleaning:
predicted_answer
a    42874
b    38378
d    26276
c    24726
e     1437
f     1396
g     1289
Name: count, dtype: int64

Sample logits array:
[25.40625  22.484375 25.484375 27.265625  0.        0.        0.      ]


In [46]:
# Check one again
a = cleaned_output[cleaned_output.label == 'e']
a.iloc[0]

question_text                         study of all aspects of planet Earth
question_image                                                         NaN
answer_choices           a. astronomy, b. oceanography, c. geology, d. ...
context_image                       textbook_images/planet_earth_20364.png
context_image_caption    FIGURE 24.1 This is how the Earth looks from s...
text_context             Earths climate has changed many times through ...
label                                                                    e
predicted_answer                                                         e
logits                   [22.03125, 22.234375, 21.65625, 24.734375, 27....
Name: 58608, dtype: object

## Upload Dataset to Huggingface Hub

In [47]:
from datasets import Dataset

In [48]:
# !huggingface-cli login
!huggingface-cli whoami

Sakonii
orgs:  talentless-us,discord-community


In [49]:
dataset = Dataset.from_pandas(cleaned_output)
dataset.push_to_hub("Sakonii/multimodal-logits-dataset", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/137 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Sakonii/multimodal-logits-dataset/commit/476ffcec6f7845c8b0d92b8e0f9e2ecbf9baf21e', commit_message='Upload dataset', commit_description='', oid='476ffcec6f7845c8b0d92b8e0f9e2ecbf9baf21e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Sakonii/multimodal-logits-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Sakonii/multimodal-logits-dataset'), pr_revision=None, pr_num=None)